#PROJECT : Analysis of Users Preferences between Android and iOS 
Here we'll implement a sentiment analysis analysisng preferences of users between Android and iOS.


In [70]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json, time, sys, io, pickle
import re
import tweepy

In [71]:
access_token = "97387398-09aWi5l77s95Vy7Jag58WvQ1soy9TzoGCA0vruWH5"
access_token_secret = "ZzBHbF3LjXg7Wl40VIa6nJomv1TdwQW9Am1oQRzIW0RLN"
consumer_key = "Ii9orQvVcYDTzbk5feDFIuz1n"
consumer_secret = "MXTqped02xYhyuer6IpO2Rt2377MRGsTHhJQnMZ9aOagGrTLnD"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


In [80]:
class TweetsListener(tweepy.StreamListener):

    def __init__(self, filename=None, api=None):
        super(TweetsListener, self).__init__()
        self.num_tweets = 0
        self.filename = filename
    
    def on_status(self, status):
        record = {'Text': status.text.encode('utf-8').strip(), 'Created At': str(status.created_at)}
        if self.num_tweets < 1000:
            try:
                with open(self.filename, 'a') as f:
                    pickle.dump(record,f)
                    self.num_tweets += 1
                return True
            except BaseException as e:
                print("Error on_status: %s" % str(e))
        else:
            return False
        
    def on_error(self, status):
        print 'Error on status', status

    def on_limit(self, status):
        print 'Limit threshold exceeded', status

    def on_timeout(self, status):
        print 'Stream disconnected; continuing...'

In [81]:
# stream = Stream(auth, TweetsListener(filename = 'data_android.json'))
# stream.filter(track=['android','androidvsios','iosvsandroid'],languages=['en'])


In [82]:
# stream = Stream(auth, TweetsListener(filename = 'data_ios.json'))
# stream.filter(track=['iphone','ios','androidvsios','iosvsandroid'],languages=['en'])

In [83]:
def readTweets(filename):
    list = []
    with open(filename, 'r') as f:
        for x in range(0,1000):
            list.append(pickle.load(f)['Text'])
    return list


In [84]:
def tokenize_with_not(text):
    """Does the same thing as tokenize_with_punct, with the following difference:
    whenever the term 'not' appears, change the two subsequent tokens to have the prefix
    'not_' prior to the token. See the example below. You may call 
    tokenize_with_punct as a subroutine.
    Params:
        text....a string
    Returns:
        a list of tokens
    """
    ###TODO
    text = re.sub(r'[\x85]',"",text)
    tokens = tokenize_with_punct(text)
    i=0
    while i< len(tokens):
        if tokens[i] == "not":
            if(i+1 < len(tokens)):
                tokens[i+1] = 'not_' + tokens[i+1]
            if(i+2 < len(tokens)):
                tokens[i+2] = 'not_' + tokens[i+2]
            i = i+3
        else:
            i = i+1
    return tokens

In [85]:
def writeListToTextFile(list,filename):
    f = open(filename, "w")
    for item in list:
      print>>f, item

In [86]:
list_android = readTweets('data_android.json')
writeListToTextFile(list_android,'data_android.txt')
list_ios = readTweets('data_ios.json')
writeListToTextFile(list_ios,'data_ios.txt')

In [87]:
def processTweet(tweet):
    # process the tweets

    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #trim
    tweet = tweet.strip('\'"')
    return tweet
#end

#Read the tweets one by one and process it


In [92]:
def preProcessTweets(filename):
    fp = open(filename, 'r')
    fpn = open('process'+filename,'w')
    line = fp.readline()
    while line:
        processedTweet = processTweet(line)
        print>>fpn, processedTweet
        line = fp.readline()
    #end loop
    fp.close()

In [93]:
preProcessTweets('data_android.txt')
preProcessTweets('data_ios.txt')